If this Colab script stop working check for updates https://github.com/danilw/Colab-shadertoy-to-video-with-FBO

___
# Instructions:
0. Look configuration in **Configuration section below**.

1. **GPU required for fast work**, but CAN be used with CPU thru `xvfb` CPU emulation (very slow).

2. On Google Colab **Press Connect** (top right)\
To upload shaders to Colab - run *Preparing* section to have shadertoy-to-video-with-FBO copy, then drop your files on path listed in Configure section.

3. Upload your **shader in order supported by** [
shadertoy-to-video-with-FBO](https://github.com/danilw/shadertoy-to-video-with-FBO) - Examples and **Shader uniforms** section there.\
Upload just by drag-drop to Colab interface - open/click _"folder icon"_ (files) on left side under `{x}` icon.\
(alternative - upload your shaders for recording to your Google drive and download them to this collab, look below in Config section)

4. **After uploading your shader(as folder)** - set `folder_name` (below in **Configuration for recording->Parameters** section)

5. **Render by pressing** - Runtime -> Run all. (after preparing comands and uploading files, **look Example**)

6. **Remember to save video result** - in Files (folder icon on left) select file and select menu to download it, *or upload it to your Google Drive*.

___

# Uploading your shader (example):

0. To **download** your shaders(as folder in zip) **from your Google drive - edit `USE_download_from_google_drive` below**.
1. [Single shader with texture example](https://github.com/danilw/shadertoy-to-video-with-FBO/tree/master/example_textures)\
Upload this folder(rename it before uploading because git clone of repository contain it) and **set Configuration below**.
2. More advanced usage - just read [shadertoy-to-video-with-FBO](https://github.com/danilw/shadertoy-to-video-with-FBO) instructions on page and look examples.


---



In [ ]:
#@title Preparing...
!pip install vispy
!pip install watchdog
!pip install Pillow
!pip install imageio

%cd /content/
!git clone https://github.com/danilw/shadertoy-to-video-with-FBO


In [ ]:
#@title Downloading your zip archive and unpacking it from your Google drive:

import os
%cd /content/

USE_download_from_google_drive = False #@param {type:"boolean"}
if(USE_download_from_google_drive):
  from google.colab import drive
  drive_path = "/content/drive"
  drive.mount(drive_path,force_remount=False)

#@markdown zip file in Google Drive, example `Colab Notebooks/<FILE_NAME>.zip`
download_zip_file = "Colab Notebooks/<FILE_NAME>.zip" #@param {type:"string"}
if(USE_download_from_google_drive):
  if(os.path.exists(drive_path+"MyDrive/"+download_zip_file)):
    path_tmp = "'"+drive_path+"MyDrive/"+download_zip_file+"'"
    fname_tmp = "'"+os.path.basename(path_tmp)+"'"
    !cp $path_tmp $fname_tmp
    !unzip $fname_tmp
    print("Unpacked:")
    !ls







---


# Configuration

In [ ]:
#@title File name of video, and Upload to Google Drive at the end

#@markdown ## Video output file name

#@markdown **Supported file extensions** - `.mp4` or `.webm` or `.mov`\
#@markdown mov is video with no compression
video_file = "1.mp4" #@param {type:"string"}

#@markdown ___

#@markdown Run this section BEFORE starting recording to connect Google Drive, if needed\
#@markdown And remember - file with similar name name will be overrited\
#@markdown Uploading location in Google Drive:\
#@markdown `Colab Notebooks/Shadertoy_video_out/<video_file>" `
Upload_to_google_drive_at_end = True #@param {type:"boolean"}
if(Upload_to_google_drive_at_end):
  from google.colab import drive
  drive_path = "/content/drive"
  drive.mount(drive_path,force_remount=False)

In [ ]:
#@title Configuration for recording

#@markdown *(all this can be skipped if you just launch your python custom comand)*

#@markdown ___

#@markdown ## Parameters

#@markdown Select to use GPU or CPU (CPU just for very simple shaders/tests, **use Colab with GPU**)
use_render = 'GPU' #@param ["GPU", "CPU"]


#@markdown Folder name with shaders, in `/content/<folder_name>`
folder_name = "shadertoy-to-video-with-FBO/example_shadertoy_fbo" #@param {type:"string"}

image_shader_name = "main_image.glsl" #@param {type:"string"}


#@markdown ___

#@markdown ## Rendering configuration

#@markdown **Video recording resolution**:\
#@markdown example `800x450` or `1280x720` or `1920x1080` or `3840x2160`
render_resolution = "800x450" #@param {type:"string"}



#@markdown Starting iTime value (seconds)
start_time = 0.0 #@param {type:"number"}



#@markdown *Only once* - number of frames to skip(render but not record), 0 is no skip.\
#@markdown `iTime==start_time` on all skipped frames, BUT iFrame will process in shader.\
#@markdown (can be good for TAA shaders)
render_and_skip_frames = 0 #@param {type:"integer"}



#@markdown Every frame - skip frames.\
#@markdown (can be good for TAA shaders)
skip_frames_every_frame = 0 #@param {type:"integer"}



#@markdown Framerate of video
frame_rate = 60 #@param {type:"integer"}



#@markdown Duration of video (seconds, support fractional part, 0.5 sec etc)
duration_video = 5 #@param {type:"number"}



#@markdown Bitrate of video (only for mp4 and webm)\
#@markdown example - `500K` to `1M` for twitter/web, `15M` for Youtube 1080p, `45M` for Youtube 4k
bitrate_video = "2M" #@param {type:"string"}


#@markdown ___
#@markdown ## Extra parameters

#@markdown ### **Tile rendering** - only for heavy *Image shader*
#@markdown Nvidia driver stop renderin if single frame takes longer than 4 sec.\
#@markdown When you render for 4k in very heavy shader - it may happen.\
#@markdown *But this parameter work only for Image shader*, Buffers rendered entire frame at once.\
#@markdown ***Keep =0 if not needed***\
#@markdown example - `128` (and remember discard do not work when tile rendering used, and dfd is `2x2`)
tile_size = 0 #@param {type:"integer"}




#@markdown ### Use **cubemap rendering**
#@markdown (remember to change `render_resolution` above to size of cube-face, to `512x512` or similar square size)\
#@markdown `panorama_rec_size` is final video size for Image shader when cubemap-rendering used\
#@markdown look "example_cubemap_recording" in shadertoy-to-video-with-FBO\
#@markdown ***Keep empty if not needed***\
#@markdown example - `1920x1080` or `3840x2160`
panorama_rec_size = "" #@param {type:"string"}

In [ ]:
#@title Printing GPU name
if(use_render=="GPU"):
  !nvidia-smi -L
else:
  !pip install glfw
  !apt update
  !apt-get install xvfb
  print("not using GPU")

In [ ]:
#@title Starting recording...
folder_name_tmp ="'"+"/content/"+folder_name+"'"
%cd $folder_name_tmp

video_file_tmp = "'"+video_file+"'"
extra_param1 = ""
extra_param2 = ""

if(len(panorama_rec_size)>2):
  extra_param1 = "--panorama_rec_size="+panorama_rec_size
if(tile_size>0):
  extra_param2 = "--tile-size="+str(tile_size)

# rendering
if(use_render=="GPU"):
  !python3 "/content/shadertoy-to-video-with-FBO/shadertoy-render.py" --output $video_file_tmp --size=$render_resolution --time=$start_time --render_and_skip_frames=$render_and_skip_frames --skip_frames_every_frame=$skip_frames_every_frame $extra_param1 $extra_param2 --rate=$frame_rate --duration=$duration_video --bitrate=$bitrate_video $image_shader_name
else:
  !xvfb-run python3 "/content/shadertoy-to-video-with-FBO/shadertoy-render.py" --vispy_use=Glfw --output $video_file_tmp --size=$render_resolution --time=$start_time --render_and_skip_frames=$render_and_skip_frames --skip_frames_every_frame=$skip_frames_every_frame $extra_param1 $extra_param2 --rate=$frame_rate --duration=$duration_video --bitrate=$bitrate_video $image_shader_name


# After finishing with rendering
# I put it here to 100% upload to Google Drive (when enabled) when cell finished
if(Upload_to_google_drive_at_end):
  from google.colab import drive
  drive_path = "/content/drive"
  drive.mount(drive_path,force_remount=False)
  folder_name_tmp ="'"+"/content/"+folder_name+"'"
  %cd $folder_name_tmp
  video_file_tmp = "'"+video_file+"'"
  dir_upl = "'"+"/content/drive/MyDrive/Colab Notebooks/Shadertoy_video_out/"+"'"
  !mkdir $dir_upl
  save_dir = "'"+"/content/drive/MyDrive/Colab Notebooks/Shadertoy_video_out/"+video_file+"'"
  !cp $video_file_tmp $save_dir
  if(len(panorama_rec_size)>2):
    %cd "/content/shadertoy-to-video-with-FBO/example_cubemap_recording/spatial-media/"
    full_vfile_name = "'"+"/content/"+folder_name+"/"+video_file_tmp+"'"
    full_out_vfile_name = "'"+"/content/"+folder_name+"/360_injected_"+video_file+"'"
    !python3 spatialmedia -i $full_vfile_name $full_out_vfile_name
    save_dir_inj = "'"+"/content/drive/MyDrive/Colab Notebooks/Shadertoy_video_out/360_injected_"+video_file+"'"
    !cp $full_out_vfile_name $save_dir_inj

  %cd "/content/"





---


# Other useful scripts:




---


## Colab related:

0. Placeholder single comand:\
You can use just single comand for recording, and ignore Configuration below.
```python
%cd "/content/shadertoy-to-video-with-FBO/example_shadertoy_fbo"
!python3 ../shadertoy-render.py --output 3.mp4 --size=800x450 --rate=30 --duration=5.0 --bitrate=5M main_image.glsl
```

1. Downloading from your Google drive: (replace `FILE_NAME`)\
(to save to Google drive - just `!cp <file> <location of mounted drive>` (just copy))
```python
from google.colab import drive
drive_path = "/content/drive"
drive.mount(drive_path,force_remount=False)
%cd /content/
#!rm -rf /content/<previous copy>
!cp "/content/drive/MyDrive/Colab Notebooks/<FILE_NAME>.zip" "<FILE_NAME>.zip"
!unzip "<FILE_NAME>.zip"
```


2. To **not disconnect from Colab**: (Colab auto disconnect if tab not active or no user input for some time)\
If you do some long recording (hours) - use this in Javascript webbrowser console(F12):
```javascript
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```


---





---



## Shaders related:

1. *shadertoy-to-video-with-FBO* use vispy python package, that **very limited in supported features** - no texture mipmaps, no cubemaps, etc...\
So if your shader needs mipmaps/cubemaps/etc - look for some other solution to make video.\
In my case, for my video recordings - I just apply **SSAA to entire shader**:
```c
void mainImage(out vec4 fragColor, in vec2 fragCoord)
{
    vec4 tcol=vec4(0.);
    const int AA=4;
    for( int mx=0; mx<AA; mx++ )
    for( int nx=0; nx<AA; nx++ )
    {
        vec2 o = vec2(float(mx),float(nx)) / float(AA) - 0.5;
        mainImage_orig(fragColor,fragCoord+o);
        tcol+=clamp(fragColor,0.,1.);
    }
    fragColor=tcol/float(AA*AA);
}
```

2. Using **video in shaders** (in *shadertoy-to-video-with-FBO* examples) - it works, enough.

3. **Cubemap recording setup** (in *shadertoy-to-video-with-FBO* examples) - it not real cubemap, it buffers.\
And if you need "cubemap feedback" (read cubemap(self) previous state from Cubemap) - do not build "cubemap" from buffers(cube-faces) because faces rendered forward and this way you will read "just rendered faces".\
To read "entire cubemap previous frame" - make new "copy buffers"(copy faces) and build cubemap from those buffers.\
Example - `Buf0-to-Buf5` is 6 cube-faces with cubemap shader code, make `Buf6-Buf11` with shader code:
```c
 #define iChannel0 u_channel<0-to-5>
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    fragColor = texelFetch(iChannel0,ivec2(fragCoord),0);
}
```
And in `Buf0-5` use `Buf6-11` to build cubemap.\
Cubemap shader functions can be found - [TEMPLATE cubemap tools(shadertoy shader)](https://www.shadertoy.com/view/NttGWr)

___






---

# Useful ffmpeg comands

1. Extrat **frames to images**:\
`ffmpeg -i video.mov -vf fps=1 "frames/out%d.png"`
2. **Video to Gif** ffmpeg commands:\
`ffmpeg -i video.mp4 -vf "fps=25,scale=640:-1:flags=lanczos" output.gif`\
or (*delay = 100/fps*)\
`ffmpeg -i video.mp4 -vf "fps=25,scale=480:-1:flags=lanczos" -c:v pam -f image2pipe - | convert -delay 4 - -loop 0 -layers optimize output.gif
`